# OLP_LLM: Pipeline for generating FOON graphs via LLM Prompting

## Step 1 :- Generate a FOON via LLM Prompting

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### Initialize library

In [7]:
from olp_lib import *

# -- use the custom-made class for accessing OpenAI models:
openai_driver = OpenAIInterfacer()

In [2]:
import utamp.driver as driver
from utamp.generate import randomize_blocks

run_utamp = False

if run_utamp:
    # -- load the scene only once:
    # utamp = driver.UTAMP(scene_file_name='./utamp/scenes/panda_blocks_tiles-5.ttt')

    fpath, tally = randomize_blocks('./utamp/scenes/panda_blocks_tiles_prototype.ttt')
    utamp = driver.UTAMP(scene_file_name=fpath)

    print("objects presently in scene:", utamp.objects_in_sim)

### Language to  OLP

In [3]:
user_tasks = [
	"How can I make a Dark and Stormy cocktail?",
	"How can I make a Bloody Mary cocktail?",
	"How can I make a Greek salad?",
	"How can I stack a tower of blocks?"
]

#### Prompt LLM for Object-level Plan (OLP):

In [8]:
# NOTE: all incontext examples will be stored within a JSON file:
# -- Q: can we randomly sample from the set of incontext examples?
# -- Q: should we also select an example "closest" to the provided task?
# incontext_file = "incontext_examples.txt"
incontext_file = "incontext_examples.json"

LLM_to_PDDL = True

user_task = choice(user_tasks)

llm_output = generate_OLP(openai_driver,
						  query=user_task,
						  incontext_file=incontext_file,
                          stage1_sys_prompt_file="./llm_prompts/stage1_system_prompt.txt",
                          stage2_sys_prompt_file="./llm_prompts/stage2_system_prompt_v2.txt",
						  verbose=False)

print('Objects referenced in plan:', llm_output['RelevantObjects'])
print('Terminal Steps:', llm_output['TerminalSteps'])

# -- save unparsed output to a JSON file:
json.dump(llm_output, open('raw_OLP.json', 'w'), indent=4)

TypeError: generate_OLP() missing 1 required positional argument: 'openai_obj'

In [ ]:
print("TASK:",  user_task)
print(f"\n{'*' * 23}\n    High level plan\n{'*' * 23}\n{llm_output['PlanSketch']}\n")

In [ ]:
print("***********************\n   Object level plan\n***********************\n")
print(llm_output['OLP'], sep="\n")
print(f"\n***********************\n   Plan objects\n***********************\n{llm_output['RelevantObjects']}\n")

#### Sample FOON creation

In [ ]:
# plan_step, plan_objects = llm_output['OLP'],

if '.json' in str(incontext_file).lower():
    sample_unit = create_functionalUnit_ver2(llm_output, index=0)
else:
    sample_unit = create_functionalUnit_ver1(llm_output['OLP'][0]['Instructions'],llm_output['RelevantObjects'])

sample_unit.print_functions[2](version=1)

#### Creating FOON units

In [ ]:
FOON_prototype = []

for x in range(len(llm_output['OLP']['Instructions'])):

    new_unit = create_functionalUnit(llm_output, index=x)

    # NOTE: in order to define a macro-problem, we need to properly identify all goal nodes;
    #       we will do this with the help of the LLM:
    if 'TerminalSteps' in llm_output and (x+1) in llm_output['TerminalSteps']:
        # -- set output objects as goal nodes for the functional units deemed as terminal steps:
        print(f'\nFunctional unit {x+1} has terminal goals!')
        for N in range(new_unit.getNumberOfOutputs()):
            new_unit.getOutputNodes()[N].setAsGoal()

    # -- add the functional unit to the FOON prototype:

    if not new_unit.isEmpty():
        # -- we should only add a new functional unit if it is not empty, meaning it must have the following:
        #    1. >=1 input node and >= 1 output node
        #    2. a valid motion node
        FOON_prototype.append(new_unit)
        FOON_prototype[-1].print_functions[-1]()
    else:
        print('NOTE: the following functional unit has an error, so skipping it:')
        new_unit.print_functions[-1]()

    print('\n', '*' * 40)

    print()


In [ ]:
if LLM_to_PDDL:

    pddl_sys_prompt_file = "llm_prompts/pddl_system_prompt.txt"

    pddl_system_prompt = open(pddl_sys_prompt_file, 'r').read()

    message = [{"role": "system", "content": pddl_system_prompt}]

    # for x in range(len(OLP_results['OLP']['Instructions'])):
    pddl_user_prompt = f"Generate PDDL for :\n{llm_output['OLP']['Instructions']}"

    message.extend([{"role": "user", "content": pddl_user_prompt}])

    _, response = openai_driver.prompt(prompt=message, verbose=False)
    print(response)


### Writing FOON to a Text File

In [ ]:
temp_file_name = 'prototype.txt'
task_file_name = f"{str.lower(re.compile('[^a-zA-Z]').sub('_', llm_output['OLP']['Query'])[:-1])}.txt"
print(task_file_name)

# -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
if not os.path.exists('./preprocess/'):
    os.makedirs('./preprocess/')

if not os.path.exists('./postprocess/'):
    os.makedirs('./postprocess/')

temp_file = open(f'./preprocess/{temp_file_name}', 'w')
temp_file.write('#\tFOON Prototype\n#\t-- Task Prompt: {0}\n//\n'.format(user_task))
for unit in FOON_prototype:
    unit.print_functions[2]()
    temp_file.write(unit.getFunctionalUnitText())
    print('//')

temp_file.close()

### Parse and clean generated FOON

In [ ]:
# -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
#		(it is important that each object and state type have a *UNIQUE* identifier)
fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
fpa.skip_index_check = True			# -- always create a new set of index files

fpa.source_dir = './preprocess/'
fpa.target_dir = './postprocess/'
fpa._run_parser()

with open(f'./postprocess/{temp_file_name}', 'r') as input_file:
    with open(f'./postprocess/{task_file_name}', 'w') as output_file:
        for line in input_file:
            output_file.write(line)

### Save generated FOON to embedding

In [ ]:
add_to_embeddings(openai_driver,
				  text=user_task,
				  func_unit=fga.FOON_functionalUnits[-1])

## Step 2 :- FOON to PDDL
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. (Optional) Visualize FOON graph

3. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### (Optional) Run FOON visualization tool

In [ ]:
# -- after running the parser, take the 'prototype.txt' file and plot it using the following tool: https://davidpaulius.github.io/foon-view/
# fga._startFOONview()

### Generate PDDL files using ```FOON_to_PDDL``` module

In [ ]:
import FOON_TAMP as fta
from pathlib import Path

FOON_subgraph_file = f'./postprocess/{task_file_name}'

# -- definition of macro and micro plan file names:
macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

fta.micro_domain_file = './FOON_skills.pddl'

fta.flag_perception = False

# -- create a new folder for the generated problem files and their corresponding plans:
fta.micro_problems_dir = './micro_problems-' + Path(FOON_subgraph_file).stem
if not os.path.exists(fta.micro_problems_dir):
    os.makedirs(fta.micro_problems_dir)

ftp = fta.ftp

# -- perform conversion of the FOON subgraph file to PDDL:
ftp.FOON_subgraph_file = FOON_subgraph_file
ftp._convert_to_PDDL('OCP')

## -- parse through the newly created domain file and find all (macro) planning operators:
all_macro_POs = {PO.name : PO for PO in fta._parseDomainPDDL(ftp.FOON_domain_file)}

# -- checking to see if a macro level plan has been found (FD allows exporting to a file):
outcome = fta._findPlan(
    domain_file=ftp.FOON_domain_file,   # NOTE: FOON_to_PDDL object will already contain the names of the
    problem_file=ftp.FOON_problem_file,  #   corresponding macro-level domain and problem files
    output_plan_file=macro_plan_file
)

# TODO: write planning pipeline for other planners?

complete_micro_plan = []

if fta._checkPlannerOutput(output=outcome):
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was found!")

    # -- now that we have a plan, we are going to parse it for the steps:
    macro_file = open(macro_plan_file, 'r')

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    macro_plan_lines = list(macro_file)
    for L in macro_plan_lines:
        if L.startswith('('):
            print('\t\t\t' + str(macro_count) + ' : ' + L.strip())
    print()

    for L in range(len(macro_plan_lines)):

        if macro_plan_lines[L].startswith('('):
            # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
            #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
            #	2. effects - this will become a sub-problem file's goal states (as predicates)

            macro_count += 1

            macro_PO_name = macro_plan_lines[L][1:-2].strip()

            print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

            # -- try to find this step's matching planning operator definition:
            matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

            # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
            if matching_PO_obj:
                # -- create sub-problem file (i.e., at the micro-level):
                micro_problem_file = fta._defineMicroProblem(
                    macro_PO_name,
                    preconditions=matching_PO_obj.getPreconditions(),
                    effects=matching_PO_obj.getEffects(),
                )

                micro_domain_file = fta._defineMicroDomain(
                    preconditions=matching_PO_obj.getPreconditions(),
                )

                complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

                need_to_replan = False

                # -- try to find a sub-problem plan / solution:
                outcome = fta._findPlan(
                    domain_file=micro_domain_file,
                    problem_file=micro_problem_file,
                    output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
                )

                print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

                if fta._checkPlannerOutput(outcome):

                    print('\t\t -- micro-level plan found as follows:')

                    # -- open the micro problem file, read each line referring to a micro PO, and save to list:
                    micro_file = open(
                        str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

                    # -- all except for the last line should be valid steps:
                    micro_file_lines = []
                    count = 0

                    for micro_line in micro_file:
                        if micro_line.startswith('('):
                            # -- parse the line and remove trailing newline character:
                            micro_step = micro_line.strip()
                            micro_file_lines.append(micro_step)

                            # -- print entire plan to the command line in format of X.Y,
                            #       where X is the macro-step count and Y is the micro-step count:
                            count += 1
                            print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                        #endif
                    #endfor
        print()
else:
    print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was NOT found!")


In [ ]:
complete_micro_plan = []

# -- we can also perform a search where we assume we already have a task tree
#		and we execute each functional unit one by one as we see

macro_plan_lines = []
for N in range(len(ftp.fga.FOON_lvl3)):
    PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
    macro_plan_lines.append(PO_name)

macro_count = 0

for L in range(len(macro_plan_lines)):

    macro_count += 1

    macro_PO_name = macro_plan_lines[L]

    print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

    # -- try to find this step's matching planning operator definition:
    matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

    # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
    if matching_PO_obj:
        # -- create sub-problem file (i.e., at the micro-level):
        micro_problem_file = fta._defineMicroProblem(
            macro_PO_name,
            preconditions=matching_PO_obj.getPreconditions(),
            effects=matching_PO_obj.getEffects(),
        )

        micro_domain_file = fta._defineMicroDomain(
            preconditions=matching_PO_obj.getPreconditions(),
        )

        complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

        need_to_replan = False

        # -- try to find a sub-problem plan / solution:
        outcome = fta._findPlan(
            domain_file=micro_domain_file,
            problem_file=micro_problem_file,
            output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
        )

        print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

        if fta._checkPlannerOutput(outcome):

            print('\t\t -- micro-level plan found as follows:')

            # -- open the micro problem file, read each line referring to a micro PO, and save to list:
            micro_file = open(
                str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

            # -- all except for the last line should be valid steps:
            micro_file_lines = []
            count = 0

            for micro_line in micro_file:
                if micro_line.startswith('('):
                    # -- parse the line and remove trailing newline character:
                    micro_step = micro_line.strip()
                    micro_file_lines.append(micro_step)

                    # -- print entire plan to the command line in format of X.Y,
                    #       where X is the macro-step count and Y is the micro-step count:
                    count += 1
                    print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                #endif
            #endfor
    print()


## Step 3: Connection with UTAMP System

In [ ]:
run_utamp = False

if run_utamp:
    # -- now that we have a plan, we are going to parse it for the steps:
    macro_file = open(macro_plan_file, 'r')

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    # macro_plan_lines = list(macro_file)
    # for L in macro_plan_lines:
    #     if L.startswith('('):
    #         print('\t' + str(macro_count) + ' : ' + L.strip())
    #         macro_count += 1

    macro_plan_lines = []
    for N in range(len(ftp.fga.FOON_lvl3)):
        PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
        # print('\t' + str(macro_count) + ' : ' + PO_name)
        macro_plan_lines.append(PO_name)

    macro_count = 0

    total_success = 0

    for L in range(len(macro_plan_lines)):
        # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
        #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
        #	2. effects - this will become a sub-problem file's goal states (as predicates)

        macro_count += 1

        # macro_PO_name = macro_plan_lines[L][1:-2].strip()
        macro_PO_name = macro_plan_lines[L]

        print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

        # -- try to find this step's matching planning operator definition:
        matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

        # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
        if matching_PO_obj:
            # -- create sub-problem file (i.e., at the micro-level):
            micro_problem_file = fta._defineMicroProblem(
                macro_PO_name,
                preconditions=matching_PO_obj.getPreconditions(),
                effects=matching_PO_obj.getEffects(),
            )

            complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')
        else:
            continue

        goal_preds, objects_in_OLP = [], []

        # -- parse the goals in the generated micro-problem file:
        micro_problem_lines = open(micro_problem_file, 'r').readlines()
        for M in range(len(micro_problem_lines)):
            if ("(:goal" in micro_problem_lines[M]):
                while True:
                    M += 1

                    if micro_problem_lines[M].startswith("))"):
                        break

                    goal_string = str(micro_problem_lines[M]).replace("\t", "").strip()
                    if goal_string:
                        # -- first, we check if a goal string line is actually referring to a comment or a negation predicate:
                        if goal_string.startswith(";") or goal_string.startswith("(not"):
                            # -- we want to skip those:
                            continue

                        # -- append the goal string to the list of goal predicates:
                        goal_preds.append(goal_string)

                        # -- we also want to parse the goal string to identify all possible expressions for objects at the object level:
                        predicate_args = goal_preds[-1][1:-1].split(" ")
                        objects_in_OLP.extend([predicate_args[x] for x in range(1, len(predicate_args))])
                #endwhile
            #endif
        #endfor

        # print("objects in OLP:", objects_in_OLP)

        assert bool(goal_preds), f"Error: empty list of goals! Check the generated micro-problem file '{micro_problem_file}'"

        # -- prompt LLMs to perform object grounding
        #       (remove any objects that do not require grounding -- these are handled by the task planner system):
        objects_in_OLP = list(set(objects_in_OLP) - set(['hand', 'air', 'table', 'robot']))

        message = [
            {
                "role": "user",
                "content": f"Assign each of these simulation objects to real-world objects. You may only assign each object once."
                    " Give your output as a Python dictionary."
                    f"\nReal-world object: {objects_in_OLP}"
                    f"\nSimulated objects: {utamp.objects_in_sim}"
                    "\nExample: {'object_1': 'sim_object_1', 'object_2': 'sim_object_2'}"
            }
        ]

        _, response = prompt_LLM(message, selected_LLM)
        # print("LLM response:", response)
        regex_matches = re.findall(r'\{.+\}', str(eval(response)))

        if bool(regex_matches):
            # -- this means that the LLM has proposed some object groundings for us:
            obj_grounding = eval(regex_matches.pop())
            print(obj_grounding)

            # -- parse the goal predicates and replace the generic object names with those of the sim objects:
            print("before grounding:", goal_preds)
            for G in range(len(goal_preds)):
                goal_pred_parts = goal_preds[G][1:-1].split(" ")
                for obj in goal_pred_parts[1:]:
                    if obj in obj_grounding:
                        goal_preds[G] = goal_preds[G].replace(obj, obj_grounding[obj])

            print("after grounding:", goal_preds)

        success = driver.main(utamp, goal_preds=goal_preds)

        total_success += int(success)

    # utamp.pause()

    print('\n\n% Successful:', total_success / macro_count * 100.0)

# Baseline: LLM-as-Planner 

In [ ]:
utamp.stop()

if run_utamp:
    utamp = driver.UTAMP(scene_file_name=fpath)

    print("objects presently in scene:", utamp.objects_in_sim)

    # -- use the LLM as a planner to acquire a task plan:

    # -- we are going to feed the LLM with the prompt of coming up with a plan using the pre-defined skills:
    llm_planner_sys_prompt = open('llm_prompts/llm_planner_system_prompt_2.txt', 'r').read()
    message = [{"role": "system", "content": llm_planner_sys_prompt}]

    llm_planner_user_prompt = f"Generate a PDDL task plan for the following task: {user_task} "\
        f"The current state of the robot's environment is provided as the following dictionary: {utamp.perform_sensing(method=-1)}."

    print(llm_planner_user_prompt)

    message.extend([{"role": "user", "content": llm_planner_user_prompt}])
    print()

    _, response = prompt_LLM(given_prompt=message, model_name=openai_models[-1], verbose=False)
    print(response)

    step_count = 0
    steps = response.split('\n')

    goals_per_step = []
    parsed_steps = []

    while True:
        step_count += 1

        found = False

        for x in range(len(steps)):
            if steps[x].startswith(f"{step_count}."):

                found = True

                step_parts = steps[x].split('(')[1].split(')')[0]
                action, args = step_parts.split(' ')[0], step_parts.split(' ')[1:]

                tokenized_step = step_parts.split(' ')

                parsed_steps.append(tokenized_step)

                if 'place' in action or 'stack' in action:
                    goals_per_step.append([
                        f"(on {args[0]} air)",
                        f"(under {args[0]} {args[1]})",
                        f"(on {args[1]} {args[0]})",
                    ])

        if not found:
            break


In [ ]:
%%script --no-raise-error false

if run_utamp:
    print(len(goals_per_step))

    total_success = 0
    for x in goals_per_step:
        success = driver.main(utamp, goal_preds=x)
        total_success += int(success)

    print('\n\n% Successful:', total_success / (step_count-1) * 100.0)

In [ ]:
if run_utamp:
    utamp.start()

    total_success = 0

    for step in parsed_steps:
        close_gripper, target_object = None, None
        if 'pick' in step[0]:
            close_gripper = 1
        else:
            close_gripper = 0
        target_object = step[-1]

        success = utamp.path_planning(
            target_object=target_object,
            gripper_action=close_gripper)

        total_success += int(success)

    print('\n\n% Successful:', total_success / (step_count-1) * 100.0)

    utamp.stop()

In [ ]:
# %%script --no-raise-error false
utamp.start()
utamp.path_planning(
	target_object='red_block_2',
	gripper_action=1,
	algorithm='PRMstar')

utamp.path_planning(
	target_object='green_block_2',
	gripper_action=0,
	algorithm='PRMstar')

utamp.path_planning(
	target_object='green_block_3',
	gripper_action=1,
	algorithm='PRMstar')

utamp.path_planning(
	target_object='red_block_2',
	gripper_action=0,
	algorithm='PRMstar')

utamp.path_planning(
	target_object='yellow_block_1',
	gripper_action=1,
	algorithm='PRMstar')

utamp.path_planning(
	target_object='yellow_tile',
	gripper_action=0,
	algorithm='PRMstar')

utamp.stop()